In [1]:
import numpy as np

In [9]:

O_hat= [[ 4.02218687e-04,  5.72678475e-03],
    [ 4.02218687e-04,  5.72678475e-03],
    [ 4.02218687e-04,  5.72678475e-03]]


TypeError: list indices must be integers or slices, not tuple

In [5]:
O_plus = O_hat[:, 0]
O_minus = O_hat[:, 1]

TypeError: list indices must be integers or slices, not tuple

In [35]:
n = 5
O_hat = np.random.rand(n, 2)
type(O_hat)

numpy.ndarray

In [36]:
print(O_hat)

[[0.12315198 0.73581229]
 [0.08148393 0.53682331]
 [0.575328   0.49937642]
 [0.90934253 0.97876421]
 [0.16252951 0.33913686]]


In [37]:
print(O_hat.transpose())

[[0.12315198 0.08148393 0.575328   0.90934253 0.16252951]
 [0.73581229 0.53682331 0.49937642 0.97876421 0.33913686]]


In [38]:
O_plue = O_hat[:,0]
print(O_plue)
O_minus = O_hat[:,1]

[0.12315198 0.08148393 0.575328   0.90934253 0.16252951]


In [39]:
matrix = [O_plue, O_minus]

In [40]:
print(matrix)

[array([0.12315198, 0.08148393, 0.575328  , 0.90934253, 0.16252951]), array([0.73581229, 0.53682331, 0.49937642, 0.97876421, 0.33913686])]


In [41]:
np.array(matrix).transpose()

array([[0.12315198, 0.73581229],
       [0.08148393, 0.53682331],
       [0.575328  , 0.49937642],
       [0.90934253, 0.97876421],
       [0.16252951, 0.33913686]])